In [1]:
# Imports

import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import glob

import scipy.signal
import holidays

from pandas.tseries.offsets import MonthEnd

import seaborn as sb

import IPython

In [2]:
# Aux Functions for Solcast and PV data processing

def get_solcast(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['PeriodEnd'])
    file = file.resample('15T').pad()
    
    return file


def get_pv(path):
    file = pd.read_csv(path)
    file.index = pd.to_datetime(file['datetime_utc'])
    file = file.resample('15T').mean()
    
    return file


def get_solcastPV(df1, df2):
    '''
    df1: PV dataframe
    df2: Solcast dataframe
    '''
    
    # Filter both dataframes for 2019 and 2020
    try:
        temp_df1 = df1['2019':'2021-04-01']
        temp_df2 = df2['2019':'2021-04-01']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data
    except:
        temp_df1 = df1['2019':'2020']
        temp_df2 = df2['2019':'2020']
        
        # Check if data is complete. If not, match the smaller indexes
        if temp_df2.shape[0] < temp_df1.shape[0]:
            last_entry = temp_df2.index
            temp_df1 = temp_df1['2019':'{}'.format(temp_df2.index[-1].tz_convert(None))]


        # Only considering 2019 and 2020 since data is complete for that period
        temp_data = pd.DataFrame({'PV': temp_df1['pv'].values}, index=temp_df1.index)
        for i in np.arange(3, len(temp_df2.columns)):
            temp_data[temp_df2.columns[i]] = temp_df2[temp_df2.columns[i]].shift(-1).values
            
        return temp_data

In [47]:
# Get data and build a dictionary for preprocessing

data = {}

folders = glob.glob('C:/Users/FEEL/Jupyter/ecgomes/upacs_study/data/*')
for folder in folders:
    # Load each of the files inside the folder
    temp_pv = get_pv('{}/pv.csv'.format(folder))
    temp_solcast = get_solcast('{}/solcast.csv'.format(folder))
    
    # Join the files into a single dataframe
    temp_upac = get_solcastPV(temp_pv, temp_solcast)
    
    temp_name = folder.split('\\')[1]
    data[temp_name] = temp_upac
    
    print('{} date range: {} - {}'.format(temp_name, temp_upac.index[0], temp_upac.index[-1]))

upac02 date range: 2019-01-01 00:00:00 - 2020-06-15 22:30:00
upac06 date range: 2019-01-01 00:00:00 - 2020-06-15 22:30:00
upac08 date range: 2019-01-01 00:00:00 - 2021-04-01 23:45:00
upac09 date range: 2019-01-01 00:00:00 - 2020-12-31 23:45:00
upac13 date range: 2019-01-01 00:00:00 - 2020-12-31 23:45:00


In [48]:
# Aux Functions for adding 2D time information

import datetime

def days_2d(df):
    '''
    Adds 2D time information for single days
    df: dataframe to add the information
    '''
    # Map the index into seconds
    timestamp_s = pd.to_datetime(df.index.values).map(datetime.datetime.timestamp)
    
    # Since we're calculating the cos and sin values from seconds, it's 60 seconds into 60 min into 24 hours per day
    day_calc = 24*60*60
    
    # Calculate the values
    dayx = np.cos((2*np.pi/day_calc) * timestamp_s)
    dayy = np.sin((2*np.pi/day_calc) * timestamp_s)
    
    return dayx, dayy
    

def years_2d(df):
    '''
    Adds 2D time representation throught a year
    df: dataframe to add the information
    '''
    # Add Year Information

    day_year = df.index.dayofyear
    year_constant = 365.2524

    yearx = np.cos((2*np.pi/year_constant) * day_year)
    yeary = np.sin((2*np.pi/year_constant) * day_year)
    
    return yearx, yeary

In [49]:
# Add the 2D time information to the data

for upac in data.keys():
    dayx, dayy = days_2d(data[upac])
    yearx, yeary = years_2d(data[upac])
    
    data[upac]['Day X'] = dayx
    data[upac]['Day Y'] = dayy
    
    data[upac]['Year X'] = yearx
    data[upac]['Year Y'] = yeary

In [50]:
# Split the data for training, validation and testing

data_train = {}
data_val = {}
data_test = {}

for upac in data.keys():
    data_train[upac] = data[upac]['2019']
    data_val[upac] = data[upac]['2020-01':'2020-03']
    data_test[upac] = data[upac]['2020-04':]

C:\Users\FEEL\AppData\Local\Temp/ipykernel_16028/2785044146.py:8: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  data_train[upac] = data[upac]['2019']


In [51]:
# Aux Function for filtering data

def filter_by_points(df, frequency='D', num_points=1440, return_dictionary=False):
    
    df_dropped = df.dropna()
    grouper = df_dropped.groupby(pd.Grouper(freq=frequency))
    
    output = 0
    if return_dictionary:
        new_dict = {}
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_dict[i[0]] = pd.DataFrame(i[1])
        output = new_dict
    else:
        new_df = pd.DataFrame({})
        for i in grouper:
            if (len(i[1]) != num_points):
                pass
            else:
                new_df = new_df.append(pd.DataFrame(i[1]))
        output = new_df
            
    return output

In [52]:
# Filter the data by number of points that should be present in a single day

filtered_train = {}
filtered_val = {}
filtered_test = {}

for upac in data_train.keys():
    filtered_train[upac] = filter_by_points(data_train[upac], frequency='D', num_points=1440/15)
    filtered_val[upac] = filter_by_points(data_val[upac], frequency='D', num_points=1440/15)
    filtered_test[upac] = filter_by_points(data_test[upac], frequency='D', num_points=1440/15)

In [53]:
# Select columns to use

USED_COLUMNS = ['PV', 
                'AirTemp', 
                'CloudOpacity',
                'Ghi',
                'GtiFixedTilt', 
                'Day Y', 'Day X',
                'Year Y', 'Year X']

In [54]:
# Data Normalization
# We don't want to normalize PV so we can capture diferences more easily

# Feature range
# PV - greater than 0
# AirTemp - Unchanged
# Cloud Opacity and Ghi - between 0 and 1
# Day X, Y and Year X and Y - already between -1 and 1

MAX_OPACITY = 100
MAX_GHI = 1023 # max value on the training set
MAX_GTI = 1071 # max value on the training set

normalized_train = {}
normalized_val = {}
normalized_test = {}

for upac in filtered_train.keys():
    normalized_train[upac] = filtered_train[upac][USED_COLUMNS].copy(deep=True)
    normalized_val[upac] = filtered_val[upac][USED_COLUMNS].copy(deep=True)
    normalized_test[upac] = filtered_test[upac][USED_COLUMNS].copy(deep=True)
    
    normalized_train[upac]['CloudOpacity'] = normalized_train[upac]['CloudOpacity'] / MAX_OPACITY
    normalized_val[upac]['CloudOpacity'] = normalized_val[upac]['CloudOpacity'] / MAX_OPACITY
    normalized_test[upac]['CloudOpacity'] = normalized_test[upac]['CloudOpacity'] / MAX_OPACITY
    
    normalized_train[upac]['Ghi'] = normalized_train[upac]['Ghi'] / MAX_GHI
    normalized_val[upac]['Ghi'] = normalized_val[upac]['Ghi'] / MAX_GHI
    normalized_test[upac]['Ghi'] = normalized_test[upac]['Ghi'] / MAX_GHI
    
    normalized_train[upac]['GtiFixedTilt'] = normalized_train[upac]['GtiFixedTilt'] / MAX_GTI
    normalized_val[upac]['GtiFixedTilt'] = normalized_val[upac]['GtiFixedTilt'] / MAX_GTI
    normalized_test[upac]['GtiFixedTilt'] = normalized_test[upac]['GtiFixedTilt'] / MAX_GTI

In [55]:
# Split the data into X and y

X_train = {}
y_train = {}

X_val = {}
y_val = {}

X_test = {}
y_test = {}

for upac in normalized_train.keys():
    trainx = normalized_train[upac].drop('PV', axis=1)
    trainy = normalized_train[upac]['PV']
    valx = normalized_val[upac].drop('PV', axis=1)
    valy = normalized_val[upac]['PV']
    testx = normalized_test[upac].drop('PV', axis=1)
    testy = normalized_test[upac]['PV']
    
    X_train[upac] = trainx
    X_val[upac] = valx
    X_test[upac] = testx
    
    y_train[upac] = trainy
    y_val[upac] = valy
    y_test[upac] = testy

In [56]:
# Import XGBoost

import xgboost as xgb

In [59]:
# Define a training loop for UPACs

from joblib import dump, load
import optuna

def train_upac(upac_name, trainx, trainy, valx, valy, testx, testy, ntrials=100, nruns=10):
    # First do a parameter sweep with Optuna
    def create_model(trial):
        # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
        sug_estimators = trial.suggest_int('n_estimators', 50, 5000)
        sug_depth = trial.suggest_int('max_depth', 10, 5000)
        sug_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e-3)
        sug_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e-3)

        sug_model = xgb.XGBRegressor(n_estimators=sug_estimators,
                                     max_depth=sug_depth,
                                     reg_alpha=sug_alpha,
                                     reg_lambda=sug_lambda)

        return sug_model


    def create_training(model):
        model.fit(trainx[upac_name], trainy[upac_name])
    
    
    def create_evaluation(model):
        temp_yhat = model.predict(valx[upac_name])
        return sklearn.metrics.mean_squared_error(valy[upac_name], temp_yhat)
    
    
    def create_objective(trial):
        # Instantiate the model
        temp_model = create_model(trial)

        # Train the model
        create_training(temp_model)

        # Evaluate model
        metrics_val = create_evaluation(temp_model)

        return metrics_val

    study = optuna.create_study(direction='minimize')
    study.optimize(create_objective, n_trials=ntrials, show_progress_bar=True)
    
    IPython.display.clear_output()
    
    
    # Then train different models using the best parameters found
    model_dictionary = {}
    for i in np.arange(nruns):
        temp_model = xgb.XGBRegressor(n_estimators=study.best_params['n_estimators'],
                                      max_depth=study.best_params['max_depth'],
                                      reg_alpha=study.best_params['reg_alpha'],
                                      reg_lambda=study.best_params['reg_lambda'])
        
        # Train the model
        temp_model.fit(trainx[upac_name],#['Ghi'].values.reshape(trainx[upac_name].values.shape[0], 1),
                       trainy[upac_name])
        
        # Save -> dump(example_model, 'example_model.joblib')
        dump(temp_model, 'models/xgboost/{}_ghi+gti/Model {:02d}.joblib'.format(upac_name, i+1))
        
        # Add it to the dictionary to return
        model_dictionary['Model {:02d}'.format(i+1)] = temp_model
        
    return study, model_dictionary

In [68]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X)
        y_pred = pd.DataFrame(y_pred, columns=['PV'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['PV'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi+gti/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name],#[['CloudOpacity', 'GtiFixedTilt', 'Day Y', 'Year X']], 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi+gti/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name],#[['CloudOpacity', 'GtiFixedTilt', 'Day Y', 'Year X']], 
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi+gti/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name],#[['CloudOpacity', 'GtiFixedTilt', 'Day Y', 'Year X']], 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [62]:
# Train UPAC08 - all features

upac08_study, upac08_models = train_upac(upac_name='upac08', 
                                         trainx=X_train, 
                                         trainy=y_train,
                                         valx=X_val, valy=y_val,
                                         testx=X_test, testy=y_test)

In [63]:
# UPAC08 best params

upac08_study.best_params

{'n_estimators': 2558,
 'max_depth': 11,
 'reg_alpha': 0.0008671596452403754,
 'reg_lambda': 0.0006235979305594}

In [64]:
# Predict UPAC08 - All features
predict_upacs(model_dictionary=upac08_models, 
              upac_name='upac08',
              X_train=X_train, y_train=y_train,
              X_val=X_val, y_val=y_val,
              X_test=X_test, y_test=y_test)

In [65]:
# Define a training loop for UPACs

from joblib import dump, load
import optuna

def train_upac_top1(upac_name, trainx, trainy, valx, valy, testx, testy, ntrials=100, nruns=10):
    # First do a parameter sweep with Optuna
    def create_model(trial):
        # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
        sug_estimators = trial.suggest_int('n_estimators', 50, 5000)
        sug_depth = trial.suggest_int('max_depth', 10, 5000)
        sug_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e-3)
        sug_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e-3)

        sug_model = xgb.XGBRegressor(n_estimators=sug_estimators,
                                     max_depth=sug_depth,
                                     reg_alpha=sug_alpha,
                                     reg_lambda=sug_lambda)

        return sug_model


    def create_training(model):
        model.fit(trainx[upac_name], trainy[upac_name])
    
    
    def create_evaluation(model):
        temp_yhat = model.predict(valx[upac_name])
        return sklearn.metrics.mean_squared_error(valy[upac_name], temp_yhat)
    
    
    def create_objective(trial):
        # Instantiate the model
        temp_model = create_model(trial)

        # Train the model
        create_training(temp_model)

        # Evaluate model
        metrics_val = create_evaluation(temp_model)

        return metrics_val

    study = optuna.create_study(direction='minimize')
    study.optimize(create_objective, n_trials=ntrials, show_progress_bar=True)
    
    IPython.display.clear_output()
    
    
    # Then train different models using the best parameters found
    model_dictionary = {}
    for i in np.arange(nruns):
        temp_model = xgb.XGBRegressor(n_estimators=study.best_params['n_estimators'],
                                      max_depth=study.best_params['max_depth'],
                                      reg_alpha=study.best_params['reg_alpha'],
                                      reg_lambda=study.best_params['reg_lambda'])
        
        # Train the model
        temp_model.fit(trainx[upac_name]['GtiFixedTilt'].values.reshape(trainx[upac_name].values.shape[0], 1),
                       trainy[upac_name])
        
        # Save -> dump(example_model, 'example_model.joblib')
        dump(temp_model, 'models/xgboost/{}_top1/Model {:02d}.joblib'.format(upac_name, i+1))
        
        # Add it to the dictionary to return
        model_dictionary['Model {:02d}'.format(i+1)] = temp_model
        
    return study, model_dictionary

In [87]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X)
        y_pred = pd.DataFrame(y_pred, columns=['PV'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['PV'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs_top1(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_top1/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name]['GtiFixedTilt'].values.reshape(-1, 1), 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_top1/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name]['GtiFixedTilt'].values.reshape(-1, 1),
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_top1/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name]['GtiFixedTilt'].values.reshape(-1, 1), 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [89]:
# Train UPAC08 - top 1

upac08_study_top1, upac08_models_top1 = train_upac_top1(upac_name='upac08', 
                                                        trainx=X_train, 
                                                        trainy=y_train,
                                                        valx=X_val, valy=y_val,
                                                        testx=X_test, testy=y_test)

In [88]:
# Predict UPAC08 - All features
predict_upacs_top1(model_dictionary=upac08_models_top1, 
                   upac_name='upac08',
                   X_train=X_train, y_train=y_train,
                   X_val=X_val, y_val=y_val,
                   X_test=X_test, y_test=y_test)

Doing training for upac08
Doing Model 01


ValueError: Feature shape mismatch, expected: 8, got 34848

#######################################

In [90]:
# Define a training loop for UPACs

from joblib import dump, load
import optuna

def train_upac_gti(upac_name, trainx, trainy, valx, valy, testx, testy, ntrials=100, nruns=10):
    # First do a parameter sweep with Optuna
    def create_model(trial):
        # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
        sug_estimators = trial.suggest_int('n_estimators', 50, 5000)
        sug_depth = trial.suggest_int('max_depth', 10, 5000)
        sug_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e-3)
        sug_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e-3)

        sug_model = xgb.XGBRegressor(n_estimators=sug_estimators,
                                     max_depth=sug_depth,
                                     reg_alpha=sug_alpha,
                                     reg_lambda=sug_lambda)

        return sug_model


    def create_training(model):
        model.fit(trainx[upac_name], trainy[upac_name])
    
    
    def create_evaluation(model):
        temp_yhat = model.predict(valx[upac_name])
        return sklearn.metrics.mean_squared_error(valy[upac_name], temp_yhat)
    
    
    def create_objective(trial):
        # Instantiate the model
        temp_model = create_model(trial)

        # Train the model
        create_training(temp_model)

        # Evaluate model
        metrics_val = create_evaluation(temp_model)

        return metrics_val

    study = optuna.create_study(direction='minimize')
    study.optimize(create_objective, n_trials=ntrials, show_progress_bar=True)
    
    IPython.display.clear_output()
    
    
    # Then train different models using the best parameters found
    model_dictionary = {}
    for i in np.arange(nruns):
        temp_model = xgb.XGBRegressor(n_estimators=study.best_params['n_estimators'],
                                      max_depth=study.best_params['max_depth'],
                                      reg_alpha=study.best_params['reg_alpha'],
                                      reg_lambda=study.best_params['reg_lambda'])
        
        # Train the model
        temp_model.fit(trainx[upac_name].drop('Ghi', axis=1),
                       trainy[upac_name])
        
        # Save -> dump(example_model, 'example_model.joblib')
        dump(temp_model, 'models/xgboost/{}_gti/Model {:02d}.joblib'.format(upac_name, i+1))
        
        # Add it to the dictionary to return
        model_dictionary['Model {:02d}'.format(i+1)] = temp_model
        
    return study, model_dictionary

In [91]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X)
        y_pred = pd.DataFrame(y_pred, columns=['PV'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['PV'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs_gti(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_gti/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name].drop('Ghi', axis=1), 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_gti/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name].drop('Ghi', axis=1),
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_gti/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name].drop('Ghi', axis=1), 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [92]:
# Train UPAC08 - Gti, no Ghi

upac08_study_gti, upac08_models_gti = train_upac_gti(upac_name='upac08', 
                                                        trainx=X_train, 
                                                        trainy=y_train,
                                                        valx=X_val, valy=y_val,
                                                        testx=X_test, testy=y_test)

In [93]:
# Predict UPAC08 - Gti
predict_upacs_gti(model_dictionary=upac08_models_gti, 
                  upac_name='upac08',
                  X_train=X_train, y_train=y_train,
                  X_val=X_val, y_val=y_val,
                  X_test=X_test, y_test=y_test)

####################################################

In [96]:
# Define a training loop for UPACs

from joblib import dump, load
import optuna

def train_upac_ghi(upac_name, trainx, trainy, valx, valy, testx, testy, ntrials=100, nruns=10):
    # First do a parameter sweep with Optuna
    def create_model(trial):
        # Do search for n_estimators, max_depth, reg_alpha and reg_lambda
        sug_estimators = trial.suggest_int('n_estimators', 50, 5000)
        sug_depth = trial.suggest_int('max_depth', 10, 5000)
        sug_alpha = trial.suggest_float('reg_alpha', 1e-5, 1e-3)
        sug_lambda = trial.suggest_float('reg_lambda', 1e-5, 1e-3)

        sug_model = xgb.XGBRegressor(n_estimators=sug_estimators,
                                     max_depth=sug_depth,
                                     reg_alpha=sug_alpha,
                                     reg_lambda=sug_lambda)

        return sug_model


    def create_training(model):
        model.fit(trainx[upac_name], trainy[upac_name])
    
    
    def create_evaluation(model):
        temp_yhat = model.predict(valx[upac_name])
        return sklearn.metrics.mean_squared_error(valy[upac_name], temp_yhat)
    
    
    def create_objective(trial):
        # Instantiate the model
        temp_model = create_model(trial)

        # Train the model
        create_training(temp_model)

        # Evaluate model
        metrics_val = create_evaluation(temp_model)

        return metrics_val

    study = optuna.create_study(direction='minimize')
    study.optimize(create_objective, n_trials=ntrials, show_progress_bar=True)
    
    IPython.display.clear_output()
    
    
    # Then train different models using the best parameters found
    model_dictionary = {}
    for i in np.arange(nruns):
        temp_model = xgb.XGBRegressor(n_estimators=study.best_params['n_estimators'],
                                      max_depth=study.best_params['max_depth'],
                                      reg_alpha=study.best_params['reg_alpha'],
                                      reg_lambda=study.best_params['reg_lambda'])
        
        # Train the model
        temp_model.fit(trainx[upac_name].drop('GtiFixedTilt', axis=1),
                       trainy[upac_name])
        
        # Save -> dump(example_model, 'example_model.joblib')
        dump(temp_model, 'models/xgboost/{}_ghi/Model {:02d}.joblib'.format(upac_name, i+1))
        
        # Add it to the dictionary to return
        model_dictionary['Model {:02d}'.format(i+1)] = temp_model
        
    return study, model_dictionary

In [95]:
# Aux Function for predicting and storing values

def do_predictions(dictionary, save_path, X, y, index):
    # Go through each model in the dictionary
    for model in dictionary.keys():
        print('Doing {}'.format(model))
        
        temp_path = '{}/{}.csv'.format(save_path, model)
        
        y_pred = dictionary[model].predict(X)
        y_pred = pd.DataFrame(y_pred, columns=['PV'],
                              index=index)
        
        y_pred.to_csv(temp_path)
        
    # Also save ground-truth data at the end of the loop
    y_true = pd.DataFrame(y, columns=['PV'],
                          index=index)
    
    temp_path_gt = '{}/gt.csv'.format(save_path)
    y_true.to_csv(temp_path_gt)
    
    
def predict_upacs_ghi(model_dictionary, upac_name, X_train, y_train, X_val, y_val, X_test, y_test):
    # Simply call the function above for each of the settings to simplify
    
    print('Doing training for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi/train'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_train[upac_name].drop('GtiFixedTilt', axis=1), 
                   y=y_train[upac_name],
                   index=normalized_train[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing validation for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi/val'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_val[upac_name].drop('GtiFixedTilt', axis=1),
                   y=y_val[upac_name],
                   index=normalized_val[upac_name].index)
    IPython.display.clear_output()
    
    print('Doing testing for {}'.format(upac_name))
    temp_path = 'results/xgboost/{}_ghi/test'.format(upac_name)
    do_predictions(dictionary=model_dictionary, 
                   save_path=temp_path, 
                   X=X_test[upac_name].drop('GtiFixedTilt', axis=1), 
                   y=y_test[upac_name],
                   index=normalized_test[upac_name].index)
    IPython.display.clear_output()

In [97]:
# Train UPAC08 - Gti, no Ghi

upac08_study_ghi, upac08_models_ghi = train_upac_ghi(upac_name='upac08', 
                                                        trainx=X_train, 
                                                        trainy=y_train,
                                                        valx=X_val, valy=y_val,
                                                        testx=X_test, testy=y_test)

In [98]:
# Predict UPAC08 - Gti
predict_upacs_ghi(model_dictionary=upac08_models_ghi, 
                  upac_name='upac08',
                  X_train=X_train, y_train=y_train,
                  X_val=X_val, y_val=y_val,
                  X_test=X_test, y_test=y_test)